# Exact Cover問題

最初にExact Cover問題について説明します。

ある自然数の集合Uを考えます。またその自然数を含むいくつかのグループ$V_{1}, V_{2}, \ldots, V_{N}$を想定します。１つの自然数が複数のグループに属していても構いません。さて、そのグループ$V_{i}$からいくつかピックアップしたときに、それらに同じ自然数が複数回含まれず、Uに含まれる自然数セットと同じになるようにピックアップする問題をExact Cover問題といいます。
さらに、選んだグループ数を最小になるようにするものを、Smallest Exact Coverといいます。

## 準備

これをwildqatを使用して解いてみます。
wildqatがインストールされていない場合は、環境に併せて以下のようにインストールしてください。
```bash
pip install wildcat
```

必要なライブラリをimportし、wildqatオブジェクトをインスタンス化します。

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import wildqat as wq

## QUBOの作成

解きたい問題のQUBOマトリクスを作成します。

最初に自然数の集合を $U = \{1, \ldots, n\}$、グループを$V_{i} \subseteq U(i=1, \ldots, N)$とします。また、i番目のグループをピックアップしたかどうかを$x_{i} \in \{1, 0\}$で表します。ピックアップされた場合は1、されなかった場合は0です。ここで、各自然数（αとします）についてピックアップされた1つのグループのみに含まれている場合に最小となるようなコスト関数$E_{A}$を考えます。

この場合、

$E_{A} = A \sum _ { \alpha = 1 } ^ { n } \left( 1 - \sum _ { i : \alpha \in V _ { i } } x _ { i } \right) ^ { 2 }$

とすると、各自然数αに対して1つのグループのみがピックアップされた場合、$E_{A} = 0$となります。

これをQUBO形式に変換していきます。まず括弧の中を展開します。

$E_{A} = A \sum _ { \alpha = 1 } ^ { n } \{ 1 - 2\sum _ { i : \alpha \in V _ { i } } x _ { i } + ( \sum _ { i : \alpha \in V _ { i } } x _ { i } ) ^ { 2 } \} $

今回$E_{A}$を最小化する問題なので、定数である{}内の第一項は無視できます。
第二項は、$x_{i} \in {1,0}$であることを利用して、次のように書き換えることができます。

$ - 2\sum _ { i : \alpha \in V _ { i } } x _ { i } = - 2\sum _ { i = j, i : \alpha \in V _ { i }, j : \alpha \in V _ { j } } x _ { i } x _ {j}$

第三項についても、i = jの場合と、$i \neq j$の場合に分けると、次の様に書き換えられます。

$ ( \sum _ { i : \alpha \in V _ { i } } x _ { i } ) ^ { 2 } = \sum _ { i = j, i : \alpha \in V _ { i }, j : \alpha \in V _ { j } } x _ { i } x _ {j} + 2 \sum _ { i \neq j, i : \alpha \in V _ { i }, j : \alpha \in V _ { j } } x _ { i } x _ {j} $

まとめると、

$E_{A} = A \sum _ { \alpha = 1 } ^ { n } ( - \sum _ { i = j, i : \alpha \in V _ { i }, j : \alpha \in V _ { j } } x _ { i } x _ {j} + 2 \sum _ { i \neq j, i : \alpha \in V _ { i }, j : \alpha \in V _ { j } } x _ { i } x _ {j} )$

となり、QUBO形式にすることができました。

In [2]:
U = [1,2,3,4,5,6,7,8,9,10]
A = 1

def get_qubo(V):
    Q = np.zeros( (len(V), len(V)) )

    for i in range(len(V)):
        for j in range(len(V)):
            for k in range(len(U)):
                alpha = U[k]
                in_Vi = V[i].count(alpha) > 0 #V[i]に存在しているか
                in_Vj = V[j].count(alpha) > 0 #V[j]に存在しているか
                if i == j and in_Vi:
                    Q[i][j] += -1
                elif i < j and in_Vi and in_Vj:
                    Q[i][j] += 2

    return Q * A

また、結果を表示する関数を定義しておきます。

In [3]:
def display_answer(list_x, energies = None, show_graph = False):
    print("Result x:", list_x)
    text = ""
    for i in range(len(list_x)):
        if(list_x[i]):
            text += str(V[i])
    print("Picked {} group(s): {}".format(sum(list_x), text))
    if energies is not None:
        print("Energy:", a.E[-1])
    if show_graph:
        plt.plot(a.E)
        plt.show()

次の通り実行してみると、正しい答えが得られていることが分かります。

In [4]:
V = [ [1,2], [3,4,5,6], [7,8,9,10], [1,3,5], [10] ]
a = wq.opt()
a.qubo = get_qubo(V)
answer = a.sa()
display_answer(answer)

2.154728889465332
Result x: [1, 1, 1, 0, 0]
Picked 3 group(s): [1, 2][3, 4, 5, 6][7, 8, 9, 10]


## Vをもう少し複雑にしてみる

Vをもう少し複雑にして（2つグループを追加して）、実行してみます。

In [7]:
V = [ [1,2], [3,4,5,6], [7,8,9,10], [1,3,5], [10], [7,9], [2,4,6,8] ]
a = wq.opt()
a.qubo = get_qubo(V)
answer = a.sa()
display_answer(answer)

2.0433428287506104
Result x: [0, 0, 0, 1, 1, 1, 1]
Picked 4 group(s): [1, 3, 5][10][7, 9][2, 4, 6, 8]


正しい答えが得られていることが分かります。

## Smallest Exact Coverへの拡張

さらにSmallest Exact Coverにも取り組んでみます。
ピックアップされる数が最小にするためには、次の$E_{B}$を考えます。

$ E _ { B } = B \sum _ { i } x _ { i } $

これも、$x_{i} \in {1,0}$であることを利用して、次のように書き換えることができます

$ E _ { B } = B \sum _ { i, j : i = j } x _ { i } x _ {j} $

そして、$E = E_{A} + E_{B}$が最小になるようにします。

In [8]:
B = A / len(U) * 0.99
def get_qubo_min(Q):
    for i in range(len(V)):
        Q[i][i] += B
    return Q

### 以前の実装で試す

まずは、この拡張が含まれていない$ E _ {A} $だけの実装したもので5回実行してみます。

結果をみると3つのグループと4つのグループがピックアップされた結果が混在しており、つねに最小数が選ばれている訳ではありません。

In [9]:
V = [ [1,2,3,4], [5,6,7,8], [9,10], [1,2], [3,4], [5,6], [7,8,9,10]]

for i in range(5):
    print("---{}回目".format(i+1))
    a = wq.opt()
    a.qubo = get_qubo(V)
    answer = a.sa()
    display_answer(answer, a.E)

---1回目
2.2751739025115967
Result x: [0, 0, 0, 1, 1, 1, 1]
Picked 4 group(s): [1, 2][3, 4][5, 6][7, 8, 9, 10]
Energy: -10.0
---2回目
1.7684478759765625
Result x: [1, 0, 0, 0, 0, 1, 1]
Picked 3 group(s): [1, 2, 3, 4][5, 6][7, 8, 9, 10]
Energy: -10.0
---3回目
1.7202730178833008
Result x: [1, 0, 0, 0, 0, 1, 1]
Picked 3 group(s): [1, 2, 3, 4][5, 6][7, 8, 9, 10]
Energy: -10.0
---4回目
1.7342917919158936
Result x: [0, 1, 1, 1, 1, 0, 0]
Picked 4 group(s): [5, 6, 7, 8][9, 10][1, 2][3, 4]
Energy: -10.0
---5回目
1.7140698432922363
Result x: [0, 0, 0, 1, 1, 1, 1]
Picked 4 group(s): [1, 2][3, 4][5, 6][7, 8, 9, 10]
Energy: -10.0


### 新しい実装で試す
$ E _ {A} + E_{B}$となっている実装で試してみます。

結果を見ると、概ね正しい答え（3つのグループ）が選ばれるようですが、まれに少しエネルギーの高い不正解（4つのグループ）の方が選ばれてしまいます。

In [12]:
for i in range(5):
    print("---{}回目".format(i+1))
    a = wq.opt()
    a.qubo = get_qubo_min(get_qubo(V))
    answer = a.sa()
    display_answer(answer, a.E)

---1回目
1.775843858718872
Result x: [1, 1, 1, 0, 0, 0, 0]
Picked 3 group(s): [1, 2, 3, 4][5, 6, 7, 8][9, 10]
Energy: -9.703
---2回目
1.897528886795044
Result x: [0, 0, 0, 1, 1, 1, 1]
Picked 4 group(s): [1, 2][3, 4][5, 6][7, 8, 9, 10]
Energy: -9.604
---3回目
1.7409799098968506
Result x: [1, 1, 1, 0, 0, 0, 0]
Picked 3 group(s): [1, 2, 3, 4][5, 6, 7, 8][9, 10]
Energy: -9.703
---4回目
1.7648088932037354
Result x: [1, 0, 0, 0, 0, 1, 1]
Picked 3 group(s): [1, 2, 3, 4][5, 6][7, 8, 9, 10]
Energy: -9.703
---5回目
1.8045849800109863
Result x: [1, 0, 0, 0, 0, 1, 1]
Picked 3 group(s): [1, 2, 3, 4][5, 6][7, 8, 9, 10]
Energy: -9.703


### 意地悪ケース
最後に意地悪なケースを試します。
{1,2}{3}{4}{5}{6}{7}{8}{9}{10}が選ばれるのが正解です。

結果を見ると、概ね正しい答えが選ばれるようですが、まれに少しエネルギーの高い不正解の方が選ばれてしまいます。

In [16]:
V = [ [1,2], [3], [4], [5], [6], [7], [8], [9], [10], [2,3,4,5,6,7,8,9,10]]
for i in range(5):
    print("---{}回目".format(i+1))
    a = wq.opt()
    a.qubo = get_qubo_min(get_qubo(V))
    answer = a.sa()
    display_answer(answer, a.E)

---1回目
2.5378057956695557
Result x: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
Picked 9 group(s): [1, 2][3][4][5][6][7][8][9][10]
Energy: -9.108999999999998
---2回目
1.8387207984924316
Result x: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
Picked 9 group(s): [1, 2][3][4][5][6][7][8][9][10]
Energy: -9.108999999999998
---3回目
2.1590309143066406
Result x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Picked 1 group(s): [2, 3, 4, 5, 6, 7, 8, 9, 10]
Energy: -8.901
---4回目
1.750368595123291
Result x: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
Picked 9 group(s): [1, 2][3][4][5][6][7][8][9][10]
Energy: -9.108999999999998
---5回目
2.4671621322631836
Result x: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
Picked 9 group(s): [1, 2][3][4][5][6][7][8][9][10]
Energy: -9.108999999999998
